# 🧩 HRM Sudoku-Extreme 1 k Demo
**Google Colab PRO (High-RAM) + T4 GPU – single-GPU reproduction of the paper’s 1 k-shot run.**  
Runtime: ~50 min on A100-high-ram, ~55 min on T4-high-ram.

In [3]:
#@title 0️⃣ Check GPU
!nvidia-smi

Sat Aug  2 12:45:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#@title 1️⃣ One-liner installs (CUDA 12.6 + PyTorch 2.4 + Flash-Attn 2)
import os, subprocess, sys
def run(cmd):
    try:
        subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("Command failed:", e.cmd)
        print("Return code:", e.returncode)
        print("Output (stdout):", e.stdout)
        print("Error (stderr):", e.stderr)
        raise # Re-raise the exception after printing

# PyTorch 2.7 + CUDA 12.6 wheels
run("pip install torch==2.7.0+cu126 --index-url https://download.pytorch.org/whl/cu126 --no-cache-dir --force-reinstall")
run("pip install torchvision==0.22.1+cu126 --index-url https://download.pytorch.org/whl/cu126 --no-cache-dir --force-reinstall")
run("pip install torchaudio==2.2.0+cu126 --index-url https://download.pytorch.org/whl/cu126 --no-cache-dir --force-reinstall")


# Ninja + setuptools for compilation
run("pip install packaging ninja wheel setuptools setuptools-scm")

# Flash-Attention 2 (works on T4/A100)
run("pip install flash-attn --no-build-isolation")

In [ ]:
#@title 2️⃣ Clone HRM repo + submodules
run("git clone --recursive https://github.com/sapientinc/HRM.git")
%cd HRM

In [ ]:
#@title 3️⃣ Python deps
run("pip install -r requirements.txt")

## 4️⃣ Build the Sudoku-Extreme 1 k dataset  
This is exactly the same as the paper’s `subsample-size 1000 --num-aug 1000`.

In [ ]:
#@title 4️⃣ Build dataset (~30 s)
run("python dataset/build_sudoku_dataset.py --output-dir data/sudoku-extreme-1k-aug-1000 --subsample-size 1000 --num-aug 1000")
!ls data/sudoku-extreme-1k-aug-1000

## 5️⃣ Train (single GPU, small batch)
We halve the batch size (192 instead of 384) to fit T4 16 GB.  
The run will auto-log to Weights & Biases if you’re logged in (`wandb login`).

In [ ]:
#@title 5️⃣ Launch training
cmd = """
OMP_NUM_THREADS=8 python pretrain.py \
  data_path=data/sudoku-extreme-1k-aug-1000 \
  epochs=2000 \
  eval_interval=500 \
  global_batch_size=192 \
  lr=7e-5 \
  puzzle_emb_lr=7e-5 \
  weight_decay=1.0 \
  puzzle_emb_weight_decay=1.0 \
  wandb_project="hrm-colab-sudoku1k"
"""
run(cmd)

## 6️⃣ Evaluate
After training finishes (~step 1500) we run the built-in exact-accuracy evaluator.

In [ ]:
#@title 6️⃣ Evaluate last checkpoint
ckpt_path = !ls -t checkpoints/*/ckpt.pt | head -1
ckpt_path = ckpt_path[0]
print("Evaluating", ckpt_path)
run(f"python evaluate.py checkpoint={ckpt_path}")

## 7️⃣ Show one solved grid
We decode the first validation sample back to a human-readable Sudoku.

In [ ]:
#@title 7️⃣ Pretty print a solved puzzle
from src.utils.sudoku import Sudoku
import torch

ckpt = torch.load(ckpt_path, map_location="cpu")
model = ckpt["model"]
model.eval()

from src.data.sudoku_dataset import SudokuDataset
ds = SudokuDataset("data/sudoku-extreme-1k-aug-1000", split="val")
sample = ds[0]

with torch.no_grad():
    logits = model(sample["input_ids"].unsqueeze(0).cuda())
pred = logits.argmax(-1).cpu()

print("Input puzzle:\n", Sudoku(sample["input_ids"].view(9,9)).grid)
print("Model solution:\n", Sudoku(pred.view(9,9)).grid)
print("Target:\n", Sudoku(sample["target"].view(9,9)).grid)

## 8️⃣ Save checkpoint to Drive (optional)
Mount your Drive and copy the 120 MB checkpoint so others can load it instantly.

In [ ]:
#@title 8️⃣ Mount Drive & save
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/hrm_sudoku1k_t4"
run(f"mkdir -p {save_dir}")
run(f"cp -r checkpoints {save_dir}")
print("Checkpoint saved to", save_dir)